In [1]:
import os
import getpass

usuario = getpass.getuser()

# os.environ['GDAL_DATA'] = f'/opt/anaconda3/envs/geoenv/share/gdal'
# os.environ['PROJ_LIB'] = f'/opt/anaconda3/envs/geoenv/share'

# en windows
os.environ['GDAL_DATA'] = f'C:\\Users\\{usuario}\\anaconda3\\envs\\advprog12\\Library\\share\\gdal'
os.environ['PROJ_LIB'] = f'C:\\Users\\{usuario}\\anaconda3\\envs\\advprog12\\Library\\share\\proj'

In [2]:
import sys

#sys.argv[1] = "C:\\Users\\angel\\Documents\\Programacion_avanzada\\practicas\\Geodatos\\" #Ángel
sys.argv[1] = "E:\\advprog\\p1\\Geodatos\\" #Edu

euc_points = sys.argv[1] + "eucaliptos_españa_puntos\\eucalyptus_points.csv"
euc_mfe = sys.argv[1] + "Eucaliptales_Cornisa_Cantabrica\\Eucaliptales_Cornisa_Cantabrica.shp"


In [3]:
import rasterio
import glob
import numpy as np
from rasterio.merge import merge
from rasterio.enums import Resampling

La siguiente celda genera una lista con las rutas de nuestras 4 imagenes que queremos transformar a multibanda.

In [13]:
#ruta1 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta2 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta3 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
#ruta4 = input("Introduzca la rutas de la carpeta que contiene las bandas en orden:")
ruta1 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\\GRANULE\\L2A_T30TUN_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta2 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\\GRANULE\\L2A_T30TUP_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta3 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\\GRANULE\\L2A_T30TVN_A002395_20250219T111258\\IMG_DATA\\R20m"
ruta4 = "S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\\GRANULE\\L2A_T30TVP_A002395_20250219T111258\\IMG_DATA\\R20m"
rutas = []
rutas.append(ruta1)
rutas.append(ruta2)
rutas.append(ruta3)
rutas.append(ruta4)
rutas

['S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\\GRANULE\\L2A_T30TUN_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\\GRANULE\\L2A_T30TUP_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\\GRANULE\\L2A_T30TVN_A002395_20250219T111258\\IMG_DATA\\R20m',
 'S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\\GRANULE\\L2A_T30TVP_A002395_20250219T111258\\IMG_DATA\\R20m']

El siguiente genera las 4 imagenes en las distintas rutas.

In [14]:
for i, ruta in enumerate(rutas):
    raster_files = sorted(glob.glob(sys.argv[1] + "imagenes/" + ruta + "/*.jp2"))
    
    if not raster_files:
        print(f"No se encontraron archivos en {ruta}")
        continue
    
    bands = []
    with rasterio.open(raster_files[0]) as src:
        meta = src.meta.copy()
        meta.update({"count": len(raster_files), "driver": "GTiff"})
    
    for file in raster_files:
        with rasterio.open(file) as src:
            bands.append(src.read(1))
    
    stacked_array = np.stack(bands)
    output_tif = sys.argv[1] + "imagenes/" + ruta + "/salida/imagen_combinada_" + str(i+1) + ".tif"
    
    with rasterio.open(output_tif, "w", **meta) as dst:
        for j in range(stacked_array.shape[0]):
            dst.write(stacked_array[j], j + 1)
    
    print(f"Archivo guardado en: {output_tif}")


Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUN_20250219T152713.SAFE\GRANULE\L2A_T30TUN_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_1.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TUP_20250219T152713.SAFE\GRANULE\L2A_T30TUP_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_2.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVN_20250219T152713.SAFE\GRANULE\L2A_T30TVN_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_3.tif
Archivo guardado en: E:\advprog\p1\Geodatos\imagenes/S2C_MSIL2A_20250219T111131_N0511_R137_T30TVP_20250219T152713.SAFE\GRANULE\L2A_T30TVP_A002395_20250219T111258\IMG_DATA\R20m/salida/imagen_combinada_4.tif


Ahora quedaría hacer el mosaico. He pensado que se puede hacer lo mismo para las rutas,
o en el bucle anterior una modificacion para que añada la ruta a otra lista. Lo haré en otro momentou